# Using different Embedding Models

Ragas allows users to change the default embedding model used in the evaluation task.

This guide will show you how to use different embedding models for evaluation in Ragas.

## Evaluating with Azure Open AI Embeddings

Ragas uses open-ai embeddings by default. In this example we can use Azure Open AI Embeddings from langchain with the embedding model text-embedding-ada-002. We will be using gpt-35-turbo-16k from Azure OpenAI as the llm for evaluation and `AnswerSimilarity` as the metric

To start-off, we initialise the gpt-35-turbo-16k from Azure and create a chat_model using langchain

In [1]:
import os
from langchain.chat_models import AzureChatOpenAI
from ragas.llms import LangchainLLM

os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["OPENAI_API_KEY"] = "your-openai-key"

azure_model = AzureChatOpenAI(
    deployment_name="your-deployment-name",
    model="gpt-35-turbo-16k",
    azure_endpoint="https://your-endpoint.openai.azure.com/",
    openai_api_type="azure",
)
# wrapper around azure_model
ragas_azure_model = LangchainLLM(azure_model)

In order to use the Azure Open AI embedding, we have to instantiate an object of the `AzureOpenAIEmbeddings` class in Ragas.

In [ ]:
from langchain.embeddings import AzureOpenAIEmbeddings

azure_embeddings = AzureOpenAIEmbeddings(
    deployment="your-deployment-name",
    model="text-embedding-ada-002",
    azure_endpoint="https://your-endpoint.openai.azure.com/",
    openai_api_type="azure",
)

To use the AzureOpenAIEmbeddings with the AnswerSimilarity metric, create an object of AnswerSimilarity by passing the azure_embeddings and llm as parameters.

In [6]:
from ragas.metrics import AnswerSimilarity

answer_similarity = AnswerSimilarity(llm=ragas_azure_model, embeddings=azure_embeddings)

That's it! answer_similarity will now be using AzureOpenAIEmbeddings under the hood for evaluations.

Now lets run the evaluations using the example from [quickstart](../../getstarted/evaluation.md).

In [4]:
# data
from datasets import load_dataset

fiqa_eval = load_dataset("explodinggradients/fiqa", "ragas_eval")
fiqa_eval

DatasetDict({
    baseline: Dataset({
        features: ['question', 'ground_truths', 'answer', 'contexts'],
        num_rows: 30
    })
})

In [7]:
# evaluate
from ragas import evaluate

result = evaluate(
    fiqa_eval["baseline"].select(range(5)),  # showing only 5 for demonstration
    metrics=[answer_similarity],
)

result

evaluating with [answer_similarity]


100%|██████████| 1/1 [00:01<00:00,  1.04s/it]


{'answer_similarity': 0.8878}

## Evaluating with FastEmbed Embeddings

`FastEmbed` is a Python library built for embedding generation and has support for popular text models. Ragas has integration with FastEmbed and can be used by instantiating an object of the FastEmbedEmbeddings class. More information regarding FastEmbed and supported models can be found [here](https://github.com/qdrant/fastembed).

In [9]:
from ragas.embeddings import FastEmbedEmbeddings

fast_embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-base-en")

100%|██████████| 252M/252M [00:10<00:00, 25.0MiB/s] 


Now lets create the metric object for AnswerSimilarity by passing the llm and embedding as the `FastEmbedEmbeddings` object that we created.

In [10]:
answer_similarity2 = AnswerSimilarity(llm=ragas_azure_model, embeddings=fast_embeddings)

Now you can run the evaluations with and analyse the results.

In [11]:
result2 = evaluate(
    fiqa_eval["baseline"].select(range(5)),  # showing only 5 for demonstration
    metrics=[answer_similarity2],
)

result2

evaluating with [answer_similarity]


100%|██████████| 1/1 [00:03<00:00,  3.26s/it]


{'answer_similarity': 0.8938}

## Evaluating with HuggingFace Embeddings

Ragas has support for using embedding models using HuggingFace. Using the `HuggingfaceEmbeddings` class in Ragas, the embedding models supported by HuggingFace can directly be used for the evaluation task.

In [ ]:
# To use embedding models from HuggingFace, you need to install the following
%pip install sentence-transformers

In [ ]:
from ragas.embeddings import HuggingfaceEmbeddings

hf_embeddings = HuggingfaceEmbeddings(model_name="BAAI/bge-small-en")

Now we follow the same steps as above to use the HuggingFace Embeddings in the ragas metrics and evaluate.

In [ ]:
answer_similarity3 = AnswerSimilarity(llm=ragas_azure_model, embeddings=hf_embeddings)

In [ ]:
result3 = evaluate(
    fiqa_eval["baseline"].select(range(5)),  # showing only 5 for demonstration
    metrics=[answer_similarity3],
)

result3

evaluating with [answer_similarity]


100%|██████████| 1/1 [00:01<00:00,  1.35s/it]


{'answer_similarity': 0.9156}